In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])
# 修改后的正确路径
train_dataset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_dataset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

数据预处理 (transforms):

    transforms.ToTensor(): 神经网络的输入必须是 PyTorch 的 Tensor 格式。这张“配方”的作用就是将输入的 PIL 图像（一种 Python 图像库格式）或 numpy 数组转换成 Tensor。同时，它会自动将图像的像素值从 [0, 255] 的范围缩放到 [0.0, 1.0] 的范围，这有助于模型训练的稳定。

    transforms.Normalize((0.5,), (0.5,)): 这是数据标准化。它会将数据的范围从 [0, 1] 变换到 [-1, 1]（计算方法是 (input - mean) / std，所以 (0-0.5)/0.5 = -1, (1-0.5)/0.5 = 1）。让数据分布在 0 附近，可以加快模型收敛速度。(0.5,) 括号里的逗号表示这是一个单元素的元组，因为 MNIST 是单通道的灰度图。

    transforms.Compose([...]): 这像一个管道，它将多个预处理步骤串联起来，按顺序执行。

数据集 (torchvision.datasets.MNIST):

    root='./data': 指定数据集下载后存放的目录。

    train=True: 表示我们加载的是训练集（60000 张图片）。train=False 表示加载测试集（10000 张图片）。

    download=True: 如果在 root 目录下找不到数据，就自动下载。非常方便。

    transform=transform: 将我们上面定义的预处理流程应用到每一张加载的图片上。

数据加载器 (DataLoader):

    为什么需要它？ 我们的训练集有 60000 张图片，不可能一次性全部加载到内存/显存中去训练，会直接爆掉。DataLoader 就是解决这个问题的。它像一个智能的“数据供给管道”。

    dataset=...: 告诉 DataLoader 从哪个数据集中取数据。

    batch_size=64: 这是批大小。DataLoader 每次不会只给我们一张图片，而是给我们一个包含 64 张图片和对应标签的“批次”。这是一种折中：既避免了单张图片训练的低效，也避免了全部数据训练的内存溢出。批处理训练也能让梯度下降更稳定。

    shuffle=True: 这个对训练集至关重要！ shuffle=True 意味着在每个训练轮次（epoch）开始时，都会打乱整个数据集的顺序。这可以防止模型学习到数据的特定顺序，增强其泛化能力。而对于测试集，我们不需要打乱顺序，所以设为 False。

In [3]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleMLP()
print(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleMLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)


SimpleMLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

model.to(device): 这是 nn.Module 提供的强大方法。它会遍历模型中所有的参数（权重和偏置），并将它们全部转移到你指定的 device（GPU 或 CPU）的内存中。
model = SimpleMLP() 就是调用类的构造函数，创建了一个我们刚刚设计的 SimpleMLP 的实例对象。
x.view(...) 是用来改变 Tensor 形状的函数。

输入 x 的原始形状是 [batch_size, 1, 28, 28] (批次大小, 通道数, 高, 宽)。

nn.Linear 层要求输入是二维的 [batch_size, in_features]。所以我们需要把 [1, 28, 28] 这几维“拍扁”成一维的 784。

-1 是一个非常方便的占位符。它告诉 PyTorch：“你帮我自动计算这个维度应该是多少”。在这里，它会自动被推断为 batch_size。所以，这行代码的作用就是把 x 的形状从 [batch_size, 1, 28, 28] 变成 [batch_size, 784]。

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

损失函数：交叉熵损失
优化器：Adam优化参数、学习率

In [5]:
print("Start training...")
num_epochs = 5

model.train()

for epoch in range(num_epochs):

    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 200 == 0:    # 每训练200个批次，打印一次平均损失
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 200:.4f}')
            running_loss = 0.0

print('Finished Training')

Start training...
Epoch [1/5], Step [200/938], Loss: 0.6812
Epoch [1/5], Step [400/938], Loss: 0.3588
Epoch [1/5], Step [600/938], Loss: 0.3170
Epoch [1/5], Step [800/938], Loss: 0.2871
Epoch [2/5], Step [200/938], Loss: 0.2330
Epoch [2/5], Step [400/938], Loss: 0.2207
Epoch [2/5], Step [600/938], Loss: 0.1998
Epoch [2/5], Step [800/938], Loss: 0.1877
Epoch [3/5], Step [200/938], Loss: 0.1669
Epoch [3/5], Step [400/938], Loss: 0.1522
Epoch [3/5], Step [600/938], Loss: 0.1389
Epoch [3/5], Step [800/938], Loss: 0.1423
Epoch [4/5], Step [200/938], Loss: 0.1285
Epoch [4/5], Step [400/938], Loss: 0.1255
Epoch [4/5], Step [600/938], Loss: 0.1230
Epoch [4/5], Step [800/938], Loss: 0.1155
Epoch [5/5], Step [200/938], Loss: 0.0966
Epoch [5/5], Step [400/938], Loss: 0.0972
Epoch [5/5], Step [600/938], Loss: 0.1074
Epoch [5/5], Step [800/938], Loss: 0.1034
Finished Training


In [6]:
model.eval()

correct = 0
total  = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 94.91%


In [7]:
import torch
import os

# 创建保存目录
save_dir = '../saved_models'
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, 'mnist_mlp_model.pth')
torch.save(model.state_dict(), model_path)
print(f"✅ 模型权重已保存到: {model_path}")

✅ 模型权重已保存到: ../saved_models/mnist_mlp_model.pth
